# SmallData.jl

Just a really simple database thing

In [1]:
@require "." Table DB db_id;

## API

First you create a database connection. Normally you would pass in a filename where the database should be stored on disk but in this case we are just creating an in memory instance

In [2]:
db = DB()

SQLite.DB(in-memory)

Next we declare a table. If the `DB` already has this table then it will use it rather than create a new one

In [3]:
ranges = Table{UnitRange{Int}}(db)

0x2 UnitRange{Int64} Table
| start | stop |
|--------------|


From here on out everything you do is done the same way you would if `ranges` was a `Vector`

In [4]:
push!(ranges, 1:3)

1x2 UnitRange{Int64} Table
| start | stop |
|--------------|
|     1 |    3 |


In [5]:
ranges[1]

1:3

And we can have multiple instances of the same table and they share their underlying storage. This means you can have several processes sharing the same data

In [6]:
push!(Table{UnitRange{Int}}(db), 6:7, 1:5, 21:22)

4x2 UnitRange{Int64} Table
| start | stop |
|--------------|
|     1 |    3 |
|     6 |    7 |
|     1 |    5 |
|    21 |   22 |


## Identity

If you inspect the type of the `ranges` table you will notice it is actually a subtype of `Table` named `ValueTable`. This is because `UnitRange`'s are immutable so we don't need to store any kind of identity along with their value. If a table contains mutable types then the identity of each row will be made available

In [7]:
db_id

WeakKeyDict{Any,Int64} with 0 entries

In [8]:
type User name::String end
users = Table{User}(db)
jake = User("Jake")
push!(users, jake)
db_id[jake]

1